Setting GPU

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
from tqdm import tqdm
import cv2
import os
import seaborn as sns
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True) 

Section 1: Run all code blocks in this section to build and train a Vanilla GAN model with pituitary dataset

In [ ]:
# load and preprocess the dataset

dataset3 = tf.keras.preprocessing.image_dataset_from_directory(directory='C:/brain_tumor/version2/classify_dataset/pituitary', label_mode=None, batch_size=8, image_size=(256, 256), shuffle=True, color_mode='grayscale')
dataset3 = dataset3.map(lambda x: (x - 127.5) / 127.5)
dataset3 = dataset3.cache()
dataset3 = dataset3.prefetch(16) 

In [ ]:
# build the generator

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, Reshape, LeakyReLU, Dropout, UpSampling2D, BatchNormalization

def generator():
    model = Sequential()
    
    model.add(Dense(16*16*256, input_dim = 256))
    model.add(LeakyReLU(0.2))
    model.add(Reshape((16,16,256)))
    
    #upsampling block 1
    model.add(UpSampling2D())
    model.add(Conv2D(128, 4, padding='same', use_bias=False))
    model.add(LeakyReLU(0.2))
    
    #upsampling block 2
    model.add(UpSampling2D())
    model.add(Conv2D(64, 4, padding='same', use_bias=False))
    model.add(LeakyReLU(0.2))
    
    #upsampling block 3
    model.add(UpSampling2D())
    model.add(Conv2D(32, 4, padding='same', use_bias=False))
    model.add(LeakyReLU(0.2))
    
    #convolutional layer, reduce to 1 channel
    model.add(UpSampling2D())
    model.add(LeakyReLU(0.2))
    model.add(Conv2D(1, 4, padding='same', use_bias=False, activation='tanh'))
    
    return model

generator = generator()
generator.summary()

In [ ]:
# generate some random noises to test the generator

image = generator.predict(np.random.randn(4, 256))
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(image):
    for i in range(4):
        ax[idx].imshow(np.squeeze(image[i]), cmap='gray')
        ax[idx].title.set_text(idx)

In [ ]:
# build the discriminator

def discriminator():
    model = Sequential()
    
    #convolutional block 1
    model.add(Conv2D(32, 4, strides=(2,2), padding='same', input_shape=(256, 256, 1)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    
    #convolutional block 2
    model.add(Conv2D(64, 4, strides=(2,2), padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    
    #convolutional block 3
    model.add(Conv2D(128, 4, strides=(2,2), padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    
    #convolutional block 4
    model.add(Conv2D(256, 4, strides=(2,2), padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    
    #flatten
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    
    return model

discriminator = discriminator()
discriminator.summary()

In [ ]:
# test the discriminator

discriminator.predict(image)

In [ ]:
# setting the optimizers and losses

from tensorflow import keras
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy
from keras.models import Model
from keras.callbacks import Callback
import os

d_optimizer=Adam(lr=0.00002)
g_optimizer=Adam(lr=0.0003)
d_loss=BinaryCrossentropy()
g_loss=BinaryCrossentropy()

In [ ]:
# training process 

class BrainTumorGAN(Model):
    def __init__(self, generator, discriminator, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.generator = generator
        self.discriminator = discriminator
    
    def compile(self, generator_optimizer, discriminator_optimizer, generator_loss, discriminator_loss, *args, **kwargs):
        super().compile(*args, **kwargs)
        
        self.generator_optimizer = generator_optimizer
        self.discriminator_optimizer = discriminator_optimizer
        self.generator_loss = generator_loss
        self.discriminator_loss = discriminator_loss
        
    def train_step(self, real_images):
        batch_size = tf.shape(real_images)[0]
        noise = tf.random.normal(shape=(batch_size, 256))
        
        # Train discriminator
        with tf.GradientTape() as discriminator_tape:
            fake_images = self.generator(noise)
            real_output = self.discriminator(real_images, training=True)
            fake_output = self.discriminator(fake_images, training=True)
            
            # Compute discriminator loss
            discriminator_real_loss = self.discriminator_loss(tf.ones_like(real_output), real_output)
            discriminator_fake_loss = self.discriminator_loss(tf.zeros_like(fake_output), fake_output)
            total_discriminator_loss = (discriminator_real_loss + discriminator_fake_loss) / 2
        
        # Backpropagation for discriminator
        discriminator_gradients = discriminator_tape.gradient(total_discriminator_loss, self.discriminator.trainable_variables)
        self.discriminator_optimizer.apply_gradients(zip(discriminator_gradients, self.discriminator.trainable_variables))
        
        # Train generator
        with tf.GradientTape() as generator_tape:
            fake_images = self.generator(noise, training=True)
            fake_output = self.discriminator(fake_images, training=True)
            
            # Compute generator loss
            generator_loss = self.generator_loss(tf.ones_like(fake_output), fake_output)
        
        # Backpropagation for generator
        generator_gradients = generator_tape.gradient(generator_loss, self.generator.trainable_variables)
        self.generator_optimizer.apply_gradients(zip(generator_gradients, self.generator.trainable_variables))
        
        return {'discriminator_loss': total_discriminator_loss, 'generator_loss': generator_loss}


In [ ]:
# create a callback after each epoch

import os
from tensorflow import keras
from keras.callbacks import Callback

class ModelCallback(Callback):
    def __init__(self, number_imgs = 3, latent_dim = 256):
        self.number_imgs = number_imgs
        self.latent_dim = latent_dim
        
    def on_epoch_end(self, epoch, logs = 'None'):
        random_latent_vectors = tf.random.uniform((self.number_imgs, self.latent_dim))
        generated_image = self.model.generator(random_latent_vectors)
        generated_image = (generated_image * 127.5) + 127.5
        generated_image.numpy()
        fig = plt.figure(figsize=(20, 20))
        for i in range(self.number_imgs):
            plt.subplot(5, 5, i+1)
            img = tf.keras.utils.array_to_img(generated_image[i])
            # img.save(os.path.join('output_dcgan2', f'generated_img_from150_{epoch}_{i}.png'))
            plt.imshow(img, cmap='gray')
            plt.axis('off')
        # plt.savefig('epoch_{:03d}.png'.format(epoch))
        plt.show()

In [ ]:
# training model

brain_tumor = BrainTumorGAN(generator, discriminator)
brain_tumor.compile(g_optimizer, d_optimizer, g_loss, d_loss)
hist = brain_tumor.fit(dataset3, epochs=200, callbacks=[ModelCallback()])

Section 2: Using trained model to generate images

In [ ]:
# call trained model

from keras.models import load_model
load_generator6 = load_model('pitui_model/gen6(1000e).h5')
load_discriminator6 = load_model('pitui_model/dis6(1000e).h5')
model6 = BrainTumorGAN(load_generator6, load_discriminator6) 
model6.compile(g_optimizer, d_optimizer, g_loss, d_loss)

In [ ]:
# generate images

import os
import tensorflow as tf
import matplotlib.pyplot as plt

number_generated_imgs = 10 # set the number of images to generate
noise = tf.random.normal([number_generated_imgs, 256])
g_img = model6.generator(noise)
g_img = (g_img * 127.5) + 127.5
g_img_numpy = g_img.numpy()
save_dir = 'C:/brain_tumor/version2/notebook/vanilla_gan/pituitary/pitui_img'
os.makedirs(save_dir, exist_ok=True)
# fig, ax = plt.subplots(ncols=4, nrows=4, figsize=(20, 20))
for i in range(number_generated_imgs):
    # ax[i // 4][i % 4].imshow(g_img_numpy[i, :, :, 0], cmap='gray') 
    # ax[i // 4][i % 4].axis('off')
    file_path = os.path.join(save_dir, f'images_14_{i}_1.png')
    tf.keras.preprocessing.image.save_img(file_path, g_img_numpy[i])
# plt.tight_layout()
# plt.show()

Section 3: Evaluate generated images by metrics

In [ ]:
# SSIM metric

import os
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim

real_images_folder = 'C:/brain_tumor/version2/notebook/classify_dataset/pituitary'
generated_images_folder = 'C:/brain_tumor/version2/notebook/vanilla_gan/pituitary/pitui_img'

def load_images_from_folder(folder):
    images = []
    image_paths = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            images.append(img)
            image_paths.append(img_path)
    return images, image_paths

real_images, real_image_paths = load_images_from_folder(real_images_folder)

generated_images, generated_image_paths = load_images_from_folder(generated_images_folder)

highest_ssim_values = []

for gen_img, gen_img_path in zip(generated_images, generated_image_paths):
    max_ssim = -1
    best_real_img_path = None
    for real_img, real_img_path in zip(real_images, real_image_paths):
        if gen_img.shape != real_img.shape:
            real_img = cv2.resize(real_img, (gen_img.shape[1], gen_img.shape[0]))

        score, _ = ssim(gen_img, real_img, full=True)
        if score > max_ssim:
            max_ssim = score
            best_real_img_path = real_img_path
    
    highest_ssim_values.append((gen_img_path, max_ssim, best_real_img_path))

# Sort the highest SSIM values in descending order
highest_ssim_values.sort(key=lambda x: x[1], reverse=True)

# Calculate the average SSIM value
total_ssim = sum(value for _, value, _ in highest_ssim_values)
average_ssim = total_ssim / len(highest_ssim_values)

for gen_img_path, value, real_img_path in highest_ssim_values:
    print(f"Generated image {gen_img_path} - Highest SSIM: {value} - Real image: {real_img_path}")

print(f"Average SSIM value: {average_ssim}")


In [ ]:
# Select generated images that have SSIM value > 0.5 
# (can store these images to another folder and apply PSNR metric to select imgs have both SSIM > 0,5 and PSNR > 30)

for value in highest_ssim_values:
    if value[1] > 0.5:
        print(value)

In [ ]:
# PSNR metric

import os
import cv2
import numpy as np

real_images_folder = 'C:/brain_tumor/version2/notebook/classify_dataset/pituitary'
generated_images_folder = 'C:/brain_tumor/version2/notebook/vanilla_gan/pituitary/pitui_img'

def load_images_from_folder(folder):
    images = []
    image_paths = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            images.append(img)
            image_paths.append(img_path)
    return images, image_paths

def compute_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    PIXEL_MAX = 255.0
    return 20 * np.log10(PIXEL_MAX / np.sqrt(mse))

real_images, real_image_paths = load_images_from_folder(real_images_folder)
generated_images, generated_image_paths = load_images_from_folder(generated_images_folder)

highest_psnr_values = []

for gen_img, gen_img_path in zip(generated_images, generated_image_paths):
    max_psnr = -1
    best_real_img_path = None
    for real_img, real_img_path in zip(real_images, real_image_paths):
        if gen_img.shape != real_img.shape:
            real_img = cv2.resize(real_img, (gen_img.shape[1], gen_img.shape[0]))

        score = compute_psnr(gen_img, real_img)
        if score > max_psnr:
            max_psnr = score
            best_real_img_path = real_img_path
    
    highest_psnr_values.append((gen_img_path, max_psnr, best_real_img_path))

# Sort the highest PSNR values in descending order
highest_psnr_values.sort(key=lambda x: x[1], reverse=True)

# Calculate the average PSNR value
total_psnr = sum(value for _, value, _ in highest_psnr_values)
average_psnr = total_psnr / len(highest_psnr_values)

for gen_img_path, value, real_img_path in highest_psnr_values:
    print(f"Generated image {gen_img_path} - Highest PSNR: {value} - Real image: {real_img_path}")

print(f"Average PSNR value: {average_psnr}")

In [ ]:
# Select generated images that have PSNR value > 30 dB

for value in highest_psnr_values:
    if value[1] > 30:
        print(value)